In [175]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as sp
from scipy.integrate import odeint
from numpy.linalg import inv
import pdb
import scipy.io

In [2]:
data_file = scipy.io.loadmat('../DATA/cooplocalization_finalproj_KFdata.mat')

In [3]:
data_file

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Apr  9 16:24:23 2020',
 '__version__': '1.0',
 '__globals__': [],
 'Qtrue': array([[0.001, 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.001, 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.01 , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.001, 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.001, 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   , 0.01 ]]),
 'Rtrue': array([[2.25e-02, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00],
        [0.00e+00, 6.40e+01, 0.00e+00, 0.00e+00, 0.00e+00],
        [0.00e+00, 0.00e+00, 4.00e-02, 0.00e+00, 0.00e+00],
        [0.00e+00, 0.00e+00, 0.00e+00, 3.60e+01, 0.00e+00],
        [0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 3.60e+01]]),
 'measLabels': array([[array(['\\gamma_{ag} (rads)'], dtype='<U18'),
         array(['\\rho_{ga} (m)'], dtype='<U13'),
         array(['\\gamma_{ga} (rads)'], dtype='<U18'),
         array(['\\xi_a (m)'], dtype='<U9'),
        

In [211]:
#Initializations

x0 = np.array([10,0,np.pi/2,-60,0,-np.pi/2]).reshape(6,1)
u = np.array([2, -np.pi/18, 12, np.pi/25])
L = 0.5
num_time_steps = 1000
delta_t = 0.1
time_steps = data_file['tvec']
Qtrue = data_file['Qtrue']
cholesky_decomposition_Qtrue = np.linalg.cholesky(Qtrue)
Rtrue = data_file['Rtrue']
cholesky_decomposition_Rtrue = np.linalg.cholesky(Rtrue)

In [228]:
#Generate True data with noise

def model(x, t, u, noise, L):
    e_g = x[0]
    n_g = x[1]
    theta_g = x[2]
    e_a = x[3]
    n_a = x[4]
    theta_a = x[5]
    v_g = u[0]
    phi_g = u[1]
    v_a = u[2]
    w_a = u[3] 
    dxdt = [0]*6
    dxdt[0] = v_g*np.cos(theta_g) + noise[0][0]
    dxdt[1] = v_g*np.sin(theta_g) + noise[1][0]
    dxdt[2] = (v_g/L)*np.tan(phi_g) + noise[2][0]
    dxdt[3] = v_a*np.cos(theta_a) + noise[3][0]
    dxdt[4] = v_a*np.sin(theta_a) + noise[4][0]
    dxdt[5] = w_a + noise[5][0]
    return dxdt

def calculate_exact_yt(x,v):
    temptop = (x[4]-x[1])
    tempbot = (x[3]-x[0])
    y1 = np.arctan2(temptop,tempbot) - (x[2])
    if y1 < 0:
        y1 = y1 + 2*np.pi
    y1 = y1 - np.pi
    y2 = ( (x[0]-x[3])**2 + (x[1]-x[4])**2 ) ** 0.5
    y3 = np.arctan2(-temptop,-tempbot) - (x[5])
    if y3 < 0:
        y3 = y3 + 2*np.pi
    y4 = x[3]
    y5 = x[4]
    
    yt = np.array([
                [y1],
                [y2],
                [y3],
                [y4],
                [y5]
                ])
    yt = yt + v
    return yt

def get_sample_noise(S):
    n = len(S)
    std_normal_covar_matrix = np.identity(n)
    std_normal_mean_matrix = np.zeros(n)
    sampled_noise_with_covar_I = np.random.multivariate_normal(std_normal_mean_matrix, std_normal_covar_matrix).reshape(n,1)
    sampled_noise_with_covar_Q = np.dot(S,sampled_noise_with_covar_I)
    return sampled_noise_with_covar_Q

def wrap_in_negative_pi_to_pi(theta):
    wrapped_angle = (theta + np.pi) % (2 * np.pi) - np.pi
    return wrapped_angle

def get_Xtrue(x0,time_steps,cholesky_decomposition_Qtrue,L):
    Xtrue = [x0]
    n = len(x0)
    for i in range(1,len(time_steps.transpose())):
        w = get_sample_noise(cholesky_decomposition_Qtrue)
        xk = Xtrue[-1]
        xk_plus1 = odeint(model, xk.reshape(n,), np.linspace(0, 0.1, 2), args = (u,w,L))
        xk_plus1 = xk_plus1[-1]
        xk_plus1[2] = wrap_in_negative_pi_to_pi(xk_plus1[2])
        xk_plus1[5] = wrap_in_negative_pi_to_pi(xk_plus1[5])
        xk_plus1 = xk_plus1.reshape(n,1)
        Xtrue.append(xk_plus1)    
    return Xtrue

def get_Ytrue(Xtrue,cholesky_decomposition_Rtrue):
    Ytrue = []
    n = len(Xtrue[0])
    for i in range(0,len(Xtrue)):
        v = get_sample_noise(cholesky_decomposition_Rtrue)
        yk = calculate_exact_yt(Xtrue[i].reshape(n,),v)
        Ytrue.append(yk)    
    return Ytrue

Xtrue = get_Xtrue(x0,time_steps,cholesky_decomposition_Qtrue,L)
Ytrue = get_Ytrue(Xtrue,cholesky_decomposition_Rtrue)

In [231]:
#Jacobians

def get_CT_Jacobians(x,u,time_update_flag,measurement_update_flag):
    
    A = 0; B = 0; Ga = 0; C = 0; D = 0
    
    if(time_update_flag):    
        A = np.array([
                [0, 0, -u[0]*np.sin(x[2]), 0, 0, 0],
                [0, 0, u[0]*np.cos(x[2]), 0, 0, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, -u[2]*np.sin(x[5])],
                [0, 0, 0, 0, 0, u[2]*np.cos(x[5])],
                [0, 0, 0, 0, 0, 0]])

        Ga = np.eye(6)
    
    if(measurement_update_flag):
    
        denominator_1 = 1 + ((x[4]-x[1])/(x[3]-x[0]))**2
        numerator_11 = (x[4]-x[1]) / ((x[3] - x[0])**2)
        C11 = numerator_11/denominator_1

        numerator_12 = -1/(x[3]-x[0])
        C12 = numerator_12/denominator_1

        C14 = -1*C11
        C15 = -1*C12

        denominator_2 = ( (x[0]-x[3])**2 + (x[1]-x[4])**2 ) ** 0.5
        C21 = (x[0]-x[3])/denominator_2
        C22 = (x[1]-x[4])/denominator_2
        C24 = -1*C21
        C25 = -1*C22

        C = np.array([
                [C11, C12, -1, C14, C15, 0],
                [C21, C22, 0, C24, C25, 0],
                [C11, C12, 0, C14, C15, -1],
                [0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 1, 0]
                ])
    
    return A,B, Ga, C, D

def get_DT_Jacobians(x,u,dt,time_update_flag,measurement_update_flag):
    
    A,B,Gamma,C,D = get_CT_Jacobians(x,u,time_update_flag,measurement_update_flag)
    F = 0; G = 0; Omega = 0; H = 0; M = 0
    if(time_update_flag):
        F = np.eye(6) + dt*A
        Omega = delta_t*Gamma
    if(measurement_update_flag):
        H = C
        M = D
    
    return F,G,Omega,H,M

In [257]:
#Extended Kalman Filter

def kf_time_update(xk,Pk,uk,Q):
    n = len(xk)
    noise = np.zeros(n).reshape(n,1)
    #Calculate Jacobians    
    Fk,Gk,Omegak,Hk,Mk = get_DT_Jacobians(xk.reshape(n,),uk,delta_t,True,False)
    #Calculate x(k+1) minus
    #Note x(k+1) minus is of shape (n,) (just one dimension)
    xk_propogated = odeint(model, xk.reshape(n,), np.linspace(0, 0.1, 2), args = (u,noise,L))
    xk_propogated = xk_propogated[-1]
    xk_propogated[2] = wrap_in_negative_pi_to_pi(xk_propogated[2])
    xk_propogated[5] = wrap_in_negative_pi_to_pi(xk_propogated[5]) 
    #Calculate P(k+1) minus
    Pk_propogated = np.dot(Fk,np.dot(Pk,Fk.transpose())) + np.dot(Omegak,np.dot(Q,Omegak.transpose()))
    
    return xk_propogated, Pk_propogated

def kf_measurement_update(xk_propogated,Pk_propogated,yk_observed,uk,R):
    p = len(yk_observed)
    #noise
    v = np.zeros(p).reshape(p,1)
    yk_predicted = calculate_exact_yt(xk_propogated,v)
    surprise_factor = yk_observed - yk_predicted
    #Calculate Jacobians
    Fk,Gk,Omegak,Hk_plus1_minus,Mk = get_DT_Jacobians(xk,uk,delta_t,False,True)
    #Calculate Kalman Gain
    fc_kalman_gain = np.dot(Pk_propogated,Hk_plus1_minus.transpose())
    sc_kalman_gain = np.dot(Hk_plus1_minus,np.dot(Pk_propogated,Hk_plus1_minus.transpose())) + R
    kalman_gain = np.dot(fc_kalman_gain,inv(sc_kalman_gain))
    
    #Calculate x(k+1) plus
    n = len(xk_propogated)
    xk_plus1 = xk_propogated.reshape(n,1) + np.dot(kalman_gain,surprise_factor)
    #Calculate P(k+1) plus
    (n,m) = Pk_propogated.shape
    I = np.identity(n)
    fc_Pk_plus1 = I - np.dot(kalman_gain,Hk_plus1_minus)
    Pk_plus1 = np.dot(fc_Pk_plus1, Pk_propogated)
    
    return xk_plus1,Pk_plus1

def get_estimated_states(x0,P0,u,observed_measurements_list,Q,R):
    xk = x0
    Pk = P0
    estimated_state_list = [xk]
    estimated_covariance_matrix_list = [Pk]
    for observed_measurement in observed_measurements_list[1:]:
        xk_propogated,Pk_propogated = kf_time_update(xk,P0,u,Q)
        xk_plus1,Pk_plus1 = kf_measurement_update(xk_propogated,Pk_propogated,observed_measurement,u,R)
        estimated_state_list.append(xk_plus1)
        estimated_covariance_matrix_list.append(Pk_plus1)
        xk = xk_plus1
        Pk = Pk_plus1

    return estimated_state_list,estimated_covariance_matrix_list

estimated_state_list,estimated_covariance_matrix_list = get_estimated_states(x0, Qtrue, u, Ytrue, Qtrue, Rtrue)

In [235]:
xk_propogated,Pk_propogated = kf_time_update(x0,Qtrue,u,Qtrue)
xk_plus1,Pk_plus1 = kf_measurement_update(xk_propogated,Pk_propogated,Ytrue[0],u,Rtrue) 

In [258]:
a,b = get_estimated_states(x0, Qtrue, u, Ytrue, Qtrue, Rtrue)

In [265]:
a[100]

array([[ 10.25051173],
       [  3.20845936],
       [  0.93370265],
       [  8.05251825],
       [-88.11766174],
       [ -0.22089587]])

In [266]:
Xtrue[100]

array([[ 10.73004206],
       [  1.86926754],
       [  0.85085895],
       [  5.75153385],
       [-90.3709343 ],
       [ -0.27025747]])

In [248]:
data_file['ydata'].transpose()[10]

array([  2.66936946,  69.98781587,   1.60416722, -58.95311551,
       -20.61898604])

In [249]:
c = [32,3245,21,1423,2]

In [250]:
c

[32, 3245, 21, 1423, 2]

In [251]:
c[1:]

[3245, 21, 1423, 2]

In [256]:
def dd( a):
    a = a*42 - a*21
    print(a)
    
a = np.array([[1],[2]])
print(a)
dd(a.reshape(2,))
print(a)

[[1]
 [2]]
[21 42]
[[1]
 [2]]


In [269]:
Ytrue[10].shape

(5, 1)